## Setup

### Packages

In [ ]:
# basic packages
import os, time, random
from itertools import product
import matplotlib.pyplot as plt

# data
import awkward as ak
import d_hep_data

# model
import m_nn
import m_lightning

# qml
import pennylane as qml
from pennylane import numpy as np

# pytorch
import torch
import torch.nn as nn
import torch.optim as optim

# pytorch_lightning
import lightning as L
import lightning.pytorch as pl

# pytorch_geometric
import networkx as nx
import torch_geometric.nn as geom_nn
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import MessagePassing

# wandb
import wandb
from lightning.pytorch.loggers import WandbLogger
wandb.login()

# reproducibility
L.seed_everything(3020616)

# faster calculation on GPU but less precision
torch.set_float32_matmul_precision("medium")

### Configurations

In [ ]:
# configuration dictionary
cf = {}
cf["time"]     = time.strftime("%Y%m%d_%H%M%S", time.localtime())
cf["wandb"]    = True
cf["project"]  = "g_3vec_2pcqnn"
cf["rnd_seed"] = None # to be determined by for loop

# data infotmation
cf["num_events"]    = "50000"
cf["sig_channel"]   = "ZprimeToZhToZinvhbb"
cf["bkg_channel"]   = "QCD_HT2000toInf"
cf["jet_type"]      = "fatjet"
cf["subjet_radius"] = None # to be determined from [0.25, 0.5, 0.75]
cf["cut_limit"]     = (500, 1500)
cf["bin"]           = 10
cf["num_bin_data"]  = None # to be determined from [100, 200, 300]

# traning configuration
cf["num_train_ratio"]   = 0.5
cf["num_test_ratio"]    = 0.5
cf["batch_size"]        = 64
cf["num_workers"]       = 0
cf["max_epochs"]        = 100
cf["accelerator"]       = "cpu"
cf["fast_dev_run"]      = False
cf["log_every_n_steps"] = cf["batch_size"] // 2

# model hyperparameters
cf["loss_function"]  = nn.BCEWithLogitsLoss()
cf["optimizer"]      = optim.Adam
cf["learning_rate"]  = 1E-3

# 2PCNN hyperparameters
cf["gnn_layers"] = None # to be determined by grid search
cf["mlp_layers"] = None # to be determined by grid search

## Data Module

In [ ]:
class JetDataModule(pl.LightningDataModule):
    def __init__(self, sig_buffer, bkg_buffer, normalize):
        '''Add a "_" prefix if it is a fastjet feature'''
        super().__init__()
        # jet events
        sig_events = sig_buffer.get_uniform_bin_data()
        bkg_events = bkg_buffer.get_uniform_bin_data()
        sig_events = self._get_pt_eta_phi(sig_events, normalize)
        bkg_events = self._get_pt_eta_phi(bkg_events, normalize)
        self.sig_data_list = self._create_data_list(sig_events, 1)
        self.bkg_data_list = self._create_data_list(bkg_events, 0)

        # count the number of training, and testing
        num_data = cf["bin"] * cf["num_bin_data"]
        assert len(self.sig_data_list) >= num_data, f"sig data not enough: {len(self.sig_data_list)} < {num_data}"
        assert len(self.bkg_data_list) >= num_data, f"bkg data not enough: {len(self.bkg_data_list)} < {num_data}"
        num_train = int(num_data * cf["num_train_ratio"])
        num_test  = int(num_data * cf["num_test_ratio"])
        print(f"DataLog: {cf['sig_channel']} has {len(self.sig_data_list)} events and {cf['bkg_channel']} has {len(self.bkg_data_list)} events.")
        print(f"Choose num_data for each channel to be {num_data} | Each channel  has num_train = {num_train}, num_test = {num_test}")

        # prepare dataset for dataloader
        train_idx = num_train
        test_idx  = num_train + num_test
        self.train_dataset = self.sig_data_list[:train_idx] + self.bkg_data_list[:train_idx]
        self.test_dataset  = self.sig_data_list[train_idx:test_idx] + self.bkg_data_list[train_idx:test_idx]

    def _get_pt_eta_phi(self, events, normalize):
        if normalize:
            f1 = np.arctan(events["_pt"] / events["pt"])
            f2 = np.pi / 2 * events["_delta_eta"]
            f3 = np.pi * events["_delta_phi"]
            arrays = ak.zip([f1, f2, f3])
        else:
            f1 = events["_pt"]
            f2 = events["_delta_eta"]
            f3 = events["_delta_phi"]
            arrays = ak.zip([f1, f2, f3])
        arrays = arrays.to_list()
        x = [torch.tensor(arrays[i], dtype=torch.float32) for i in range(len(arrays))]
        return x
    
    def _create_data_list(self, events, y):
        # create pytorch_geometric "Data" object
        data_list = []
        for i in range(len(events)):
            x = events[i]
            edge_index = list(product(range(len(x)), range(len(x))))
            edge_index = torch.tensor(edge_index).transpose(0, 1)
            x.requires_grad, edge_index.requires_grad = False, False
            data_list.append(Data(x=x, edge_index=edge_index, y=y))
        random.shuffle(data_list)
        return data_list

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=cf["batch_size"], num_workers=cf["num_workers"],  shuffle=True)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=cf["batch_size"], num_workers=cf["num_workers"])

## Models

In [ ]:
class PCNNMessagePassing(MessagePassing):
    def __init__(self, phi, gamma, aggr):
        super().__init__(aggr=aggr, flow="target_to_source")
        self.phi   = phi
        self.gamma = gamma
    def forward(self, x, edge_index):
        return self.propagate(edge_index, x=x)
    def message(self, x_i, x_j):
        return self.phi(torch.cat((x_i, x_j), dim=-1))
    def update(self, aggr_out, x):
        # only use pt and eta in gamma
        return self.gamma(torch.cat((x[..., :2], aggr_out), dim=-1))

class Classical2PCNNForward(nn.Module):
    def __init__(self, gnn_in, gnn_layers, gnn_aggr, mlp_in, mlp_out, mlp_hidden, mlp_layers):
        super().__init__()
        # GNN
        num_features   = 3 # pt, eta, phi
        in_channel     = num_features * 2 # 2 from 2PCNN
        hidden_channel = 
        gnn_phi      = m_nn.ClassicalMLP(in_channel=in_channel, hidden_channel=)
        gnn_gamma = 
        self.gnn  = m_nn.CatMessagePassing()


        self.gnn = Classical2PCNNForwardMP(gnn_in, gnn_layers, gnn_aggr)
        self.mlp = ClassicalMLP(mlp_in, mlp_out, mlp_hidden, mlp_layers)
    def forward(self, x, edge_index, batch):
        x = self.gnn(x, edge_index)
        x = geom_nn.global_mean_pool(x, batch)
        x = self.mlp(x)
        return x

class Quantum2PCQNNForward(nn.Module):
    def __init__(self, gnn_in, gnn_layers, gnn_reupload, gnn_aggr, mlp_in, mlp_out, mlp_hidden, mlp_layers):
        super().__init__()
        num_features   = 3 # pt, eta, phi
        in_channel     = num_features * 2 # 2 from 2PCNN
        gnn_phi = m_nn.QuantumMLP(num_qubits=gnn_in)
        self.gnn = Quantum2PCQNNForwardMP(gnn_in, gnn_layers, gnn_reupload, gnn_aggr)
        self.mlp = ClassicalMLP(mlp_in, mlp_out, mlp_hidden, mlp_layers)
    def forward(self, x, edge_index, batch):
        x = self.gnn(x, edge_index)
        x = geom_nn.global_mean_pool(x, batch)
        x = self.mlp(x)
        return x

## Train/Test the Model

### Training procedure

In [ ]:
def train(model, data_module, commit="", suffix=""):
    # setup id and path for saving
    project  = cf['project']
    group    = f"{cf['time']}_{commit}_{cf['sig_channel']}_{cf['bkg_channel']}_{cf['jet_type']}"
    job_type = f"R{cf['subjet_radius']}_B{cf['bin']}_D{cf['num_bin_data']}"
    name     = f"{model.__class__.__name__}_gl{cf['gnn_layers']}_ml{cf['mlp_layers']}_{suffix} | {job_type} | rnd_{cf['rnd_seed']} | {cf['time']}"
    id       = f"{name}"
    tags     = [model.__class__.__name__, cf['sig_channel'], cf['bkg_channel'], cf['jet_type'], str(cf['subjet_radius'])]
    root_dir = f"./result"
    if not os.path.isdir(root_dir):
        os.makedirs(root_dir)

    # check whether wandb name exists
    api     = wandb.Api()
    project = cf["project"]
    if id in set([run.id for run in api.runs(f"ntuyianchen/{project}")]):
        print(f"{id} already exists, skip this run")
        return

    # wandb logger setup
    if cf["wandb"]:
        cf["model_structure"] = f"gl{cf['gnn_layers']}_ml{cf['mlp_layers']}"
        cf["model_name"]      = model.__class__.__name__
        cf["group_rnd_seed"]  = f"{cf['model_name']}_{cf['model_structure']}_{suffix} | {job_type}"
        cf["suffix"]          = suffix
        wandb_logger = WandbLogger(project=project, group=group, job_type=job_type, name=name, id=id, save_dir=root_dir, tags=tags)
        wandb_logger.experiment.config.update(cf)
        wandb_logger.watch(model, log="all")

    # start lightning training
    logger   = wandb_logger if cf["wandb"] else None
    trainer  = L.Trainer(
        logger=logger, 
        accelerator       = cf["accelerator"],
        max_epochs        = cf["max_epochs"],
        fast_dev_run      = cf["fast_dev_run"],
        log_every_n_steps = cf["log_every_n_steps"],
        )
    litmodel = LitModel(model)
    trainer.fit(litmodel, datamodule=data_module)
    trainer.test(litmodel, datamodule=data_module)

    # finish wandb monitoring
    if cf["wandb"]:
        wandb.finish()

### Start training each model

In [ ]:
def grid_train(sig_buffer, bkg_buffer, commit=""):
    # setup
    L.seed_everything(cf["rnd_seed"])

    # data module
    data_pt_eta_phi = JetDataModule(sig_buffer, bkg_buffer, events_func=events_pt_eta_phi, norm=False)
    data_pt_eta_phi_norm = JetDataModule(sig_buffer, bkg_buffer, events_func=events_pt_eta_phi, norm=True)

    # classical 2pcnn
    input_dim = data_pt_eta_phi.train_dataset[0].x.shape[1]
    cf_2pcnn = {
        "gnn_in":input_dim, 
        "gnn_layers":cf["gnn_layers"],
        "gnn_aggr":"add", 
        "mlp_in":input_dim,
        "mlp_out":1, 
        "mlp_hidden":3*input_dim, 
        "mlp_layers":cf["mlp_layers"],
    }
    train(Classical2PCNNForward(**cf_2pcnn), data_pt_eta_phi, commit, suffix=f"pep")
    train(Classical2PCNNForward(**cf_2pcnn), data_pt_eta_phi_norm, commit, suffix=f"pep_norm")

    # quantum 2pcqnn
    input_dim = data_pt_eta_phi_norm.train_dataset[0].x.shape[1]
    cf_2pcqnn = {
        "gnn_in":input_dim,
        "gnn_layers":cf["gnn_layers"],
        "gnn_reupload":input_dim,
        "gnn_aggr":"add",
        "mlp_in":input_dim,
        "mlp_out":1,
        "mlp_hidden":3*input_dim,
        "mlp_layers":cf["mlp_layers"],
    }
    train(Quantum2PCQNNForward(**cf_2pcqnn), data_pt_eta_phi_norm, commit, suffix=f"pep_norm")

In [ ]:
# use old date to keep run break runs
old_time = input("Input old time if needed else just press enter: ")
if old_time != "" and old_time != None:
    cf["time"] = old_time

# short description of this experiment
commit = input("Commit of this experiment (short description): ")

print(f"Date = {cf['time']} | Commit = {commit}")
input("Press enter to continue")
for subjet_radius in [0.75, 0.5, 0.25]:
    for num_bin_data in [50, 100]:
        cf["subjet_radius"] = subjet_radius
        cf["num_bin_data"]  = num_bin_data
        arg_buffer = [cf["num_events"], cf["jet_type"], cf["subjet_radius"], cf["cut_limit"], cf["bin"], cf["num_bin_data"]]
        sig_buffer = d_hep_data.UniformBinJetBuffer(cf["sig_channel"], *arg_buffer)
        bkg_buffer = d_hep_data.UniformBinJetBuffer(cf["bkg_channel"], *arg_buffer)
        for gl, ml in [(1,2), (1,4)]:
            cf["gnn_layers"] = gl
            cf["mlp_layers"] = ml
            for rnd_seed in range(5):
                cf["rnd_seed"] = rnd_seed
                grid_train(sig_buffer, bkg_buffer, commit)